# Convert the SWMM model values of DWF to the WEST model 

In [1]:

import SWMMToWESTConvert.aggregateNetwork as agN
import SWMMToWESTConvert.updateWESTfiles as uw


In [2]:
#MODEL points for checking the flow
LIMOILOU_COL= "RA_180174" #"DOM_941798" pipe
PASCAL_COL = "U89A" #"U004" pipe
STSACRA_COL = "R689587" #"UNI_602608" pipe
BEAU_COL = 'Pompe-Beauport'
NOEST_COL= "R016595" #'U003_Nord_Ouest' pipe
linkMeasurementFlow = [LIMOILOU_COL,BEAU_COL,STSACRA_COL,NOEST_COL,PASCAL_COL] #Change this to the nodes within the path

WTP_TANK = "RA_606859"
ST_SACR_LEAF_NODE= "R006831" #"R007637" previous assumption (and wrong)


# Get pipe sections from detailed model ------------------------

In [3]:
file = 'DWF2022.inp' 

#Links measurement are assumed to be discharging into the trunk path (not in it)
trunkModels, branchesModels = agN.aggregateAndModelNetwork(file, WTP_TANK, linkMeasurementFlow,ST_SACR_LEAF_NODE)

------------------------Getting the elements from the network ------------------------------------------------
There were 17 subcatchments with area 0
There were 0 DWFs with inflow type different than FLOW
There were 0 DFs with other constituents than FLOW
-------------------------------Obtaining and modelling the Trunk -------------------------------------------------
There are 22 connections to the path
1  relevant branches
15  connections to the path to be converted into catchments
4  points to break the path were found
Number of resulting sections of the path 5
----------------------------
Final number of pipe sections  5
Final number of catchments:  9
Final number of tanks in series: 35
-------------------------------Obtaining and converting the branches --------------------------------------------------
--------------------------Obtaining and modelling branch  U004 -------------------------------------------
There are 12 connections to the path
2  relevant branches
10  connection

In [3]:
branchesModels

NameError: name 'branchesModels' is not defined

# Start modifying WEST MODEL --------------------------

In [6]:
import SWMMToWESTConvert.SWMMtoWESTConstants as STW_C
from SWMMToWESTConvert import updateWESTfiles as uf

connProps = {}
connProps[STW_C.VEL_MIN_CONN] = 18
connProps[STW_C.VEL_CLASSES_CONN]=[23.6,47.2,65.8,141.5,283,565.9,800,943.2,1800,2829.6]

connectorsDict = {}
for b, indexes in zip(['Trunk','DOM_1631422'],[range(1,10),range(1,7)]):
    connectors= []
    for c in indexes:
        connectors.append(connProps)
    connectorsDict[b] = connectors


#classes to be assigned to the west blocks by model
modelClasses = {STW_C.SEWER_CLASS: 'psvdmSewerTank_Classplus_decay',
                STW_C.CATCH_CLASS: 'Catchment_NoRetention_IndirectStormwater',
                STW_C.CONN_CLASS: 'Kosim1ToPSVDM_curve_tracerflow_ClassplusViral',
                STW_C.COMB_CLASS: 'psvdmTwoCombiner'}


#files 
layoutFile = '01-Data/05-OneBranch157Tanks/OneBranch157Tanks.Layout.xml'
layoutFileMod = '02-Output/05-OneBranch157Tanks/OneBranch157TanksMOD.Layout.xml'


In [7]:
uf.updateWESTLayoutFile(layoutFile, layoutFileMod, modelClasses, trunkModels, branchesModels, connectorsDict)

Number of submodels found  238
The number of sewers found were  192 , catchments  0 , connectors  0  and combiners  16


AssertionError: The number of combiners is not equal to the number of catchments plus the number of branches

# Find trunk pipe from entire network

In [ ]:
import SWMMToWESTConvert.findPaths as fp
import SWMMToWESTConvert.getNetworkFromSWMM as gnfs

file = 'DWF2022.inp' 

linksN = gnfs.getsNetworksLinks(file)

outFile = gnfs.getSimulationResultsFile(file)

trunkCalculatedDF= fp.findMainFlowPath(WTP_TANK,outFile,linksN)

TESTS!!

In [ ]:

import SWMMToWESTConvert.getNetworkFromSWMM as gnfs
import swmmio

file = 'DWF2022.inp' 

model = swmmio.Model(file)

dflows, dflowsTimeseries  = gnfs.getWaterDirectFlows(model)

display(dflows)
display(dflowsTimeseries)

There were  0 DFs with other constituents than FLOW


,Time Series,Sfactor,Baseline
Node,,,
1_144,NaN,1,0.003000
1_156,NaN,1,0.003453
1_198,NaN,1,0.001100
1_210,NaN,1,0.009281
1_218,NaN,1,0.005200
...,...,...,...
RA_11071,NaN,1,0.000001
RA_14614,NaN,1,0.000001
RA_207381,NaN,1,0.000001


,Date,Time,Value
Name,,,
